In [1]:
import torch
from diffusers import StableAudioPipeline

device = "cuda:2" if torch.cuda.is_available() else "cpu"
pipe = StableAudioPipeline.from_pretrained(
    "stabilityai/stable-audio-open-1.0", torch_dtype=torch.float16
)
pipe = pipe.to(device)

/home/ndrw1221/miniconda3/envs/finetune-stable-audio-open/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]/home/ndrw1221/miniconda3/envs/finetune-stable-audio-open/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Loading pipeline components...: 100%|██████████| 6/6 [00:04<00:00,  1.37it/s]


In [3]:
pipe.components.keys()

dict_keys(['vae', 'text_encoder', 'projection_model', 'tokenizer', 'transformer', 'scheduler'])

In [26]:
transformer = pipe.transformer
transformer.requires_grad_(False)
for block in transformer.transformer_blocks:
    block.attn2.to_k.weight.requires_grad = True
    block.attn2.to_v.weight.requires_grad = True

In [27]:
for name, param in transformer.named_parameters():
    if param.requires_grad:
        print(name, param.shape)

transformer_blocks.0.attn2.to_k.weight torch.Size([768, 768])
transformer_blocks.0.attn2.to_v.weight torch.Size([768, 768])
transformer_blocks.1.attn2.to_k.weight torch.Size([768, 768])
transformer_blocks.1.attn2.to_v.weight torch.Size([768, 768])
transformer_blocks.2.attn2.to_k.weight torch.Size([768, 768])
transformer_blocks.2.attn2.to_v.weight torch.Size([768, 768])
transformer_blocks.3.attn2.to_k.weight torch.Size([768, 768])
transformer_blocks.3.attn2.to_v.weight torch.Size([768, 768])
transformer_blocks.4.attn2.to_k.weight torch.Size([768, 768])
transformer_blocks.4.attn2.to_v.weight torch.Size([768, 768])
transformer_blocks.5.attn2.to_k.weight torch.Size([768, 768])
transformer_blocks.5.attn2.to_v.weight torch.Size([768, 768])
transformer_blocks.6.attn2.to_k.weight torch.Size([768, 768])
transformer_blocks.6.attn2.to_v.weight torch.Size([768, 768])
transformer_blocks.7.attn2.to_k.weight torch.Size([768, 768])
transformer_blocks.7.attn2.to_v.weight torch.Size([768, 768])
transfor

In [5]:
pipe.vae.requires_grad_(False)
pipe.text_encoder.requires_grad_(False)
pipe.projection_model.requires_grad_(False)

StableAudioProjectionModel(
  (text_projection): Identity()
  (start_number_conditioner): StableAudioNumberConditioner(
    (time_positional_embedding): Sequential(
      (0): StableAudioPositionalEmbedding()
      (1): Linear(in_features=257, out_features=768, bias=True)
    )
  )
  (end_number_conditioner): StableAudioNumberConditioner(
    (time_positional_embedding): Sequential(
      (0): StableAudioPositionalEmbedding()
      (1): Linear(in_features=257, out_features=768, bias=True)
    )
  )
)

In [8]:
for name, param in pipe.transformer.named_parameters():
    if param.requires_grad:
        print(name)

transformer_blocks.0.attn1.to_q.lora_A.default.weight
transformer_blocks.0.attn1.to_q.lora_B.default.weight
transformer_blocks.0.attn1.to_k.lora_A.default.weight
transformer_blocks.0.attn1.to_k.lora_B.default.weight
transformer_blocks.0.attn1.to_v.lora_A.default.weight
transformer_blocks.0.attn1.to_v.lora_B.default.weight
transformer_blocks.0.attn1.to_out.0.lora_A.default.weight
transformer_blocks.0.attn1.to_out.0.lora_B.default.weight
transformer_blocks.0.attn2.to_q.lora_A.default.weight
transformer_blocks.0.attn2.to_q.lora_B.default.weight
transformer_blocks.0.attn2.to_k.lora_A.default.weight
transformer_blocks.0.attn2.to_k.lora_B.default.weight
transformer_blocks.0.attn2.to_v.lora_A.default.weight
transformer_blocks.0.attn2.to_v.lora_B.default.weight
transformer_blocks.0.attn2.to_out.0.lora_A.default.weight
transformer_blocks.0.attn2.to_out.0.lora_B.default.weight
transformer_blocks.1.attn1.to_q.lora_A.default.weight
transformer_blocks.1.attn1.to_q.lora_B.default.weight
transformer_

In [7]:
from peft import get_peft_model, LoraConfig

# transformer 加上 LoRA
lora_config = LoraConfig(
    r=4,  # bottleneck rank
    lora_alpha=16,  # scaling factor
    target_modules=[
        "to_q",
        "to_k",
        "to_v",
        "to_out.0",
    ],
    lora_dropout=0.1,
    bias="none",
)

# 取得 transformer 模型，加入 LoRA
transformer_with_lora = get_peft_model(pipe.transformer, lora_config)

In [ ]:
from safetensors.torch import load_file

# 載入 safetensors 檔案
tensor_dict = load_file(
    "/mnt/gestalt/home/ndrw1221/sao_pili-output/output/run_2025-05-23 03:08:48/checkpoint-5000/model.safetensors"
)

# 印出有哪些 tensor、各自的 shape 和 dtype
for key, tensor in tensor_dict.items():
    print(f"{key}: shape={tuple(tensor.shape)}, dtype={tensor.dtype}")

base_model.model.cross_attention_proj.0.weight: shape=(768, 768), dtype=torch.float16
base_model.model.cross_attention_proj.2.weight: shape=(768, 768), dtype=torch.float16
base_model.model.global_proj.0.weight: shape=(1536, 1536), dtype=torch.float16
base_model.model.global_proj.2.weight: shape=(1536, 1536), dtype=torch.float16
base_model.model.postprocess_conv.weight: shape=(64, 64, 1), dtype=torch.float16
base_model.model.preprocess_conv.weight: shape=(64, 64, 1), dtype=torch.float16
base_model.model.proj_in.weight: shape=(1536, 64), dtype=torch.float16
base_model.model.proj_out.weight: shape=(64, 1536), dtype=torch.float16
base_model.model.time_proj.weight: shape=(128,), dtype=torch.float16
base_model.model.timestep_proj.0.bias: shape=(1536,), dtype=torch.float16
base_model.model.timestep_proj.0.weight: shape=(1536, 256), dtype=torch.float16
base_model.model.timestep_proj.2.bias: shape=(1536,), dtype=torch.float16
base_model.model.timestep_proj.2.weight: shape=(1536, 1536), dtype=to